## Collaborative Filtering Based Recommendation System

Collaborative filtering is based on the idea that similar people (based on the data) generally tend to like similar things. It predicts which item a user will like based on the item preferences of other similar users. 

Collaborative filtering uses a user-item matrix to generate recommendations. This matrix contains the values that indicate a user’s preference towards a given item. These values can represent either explicit feedback (direct user ratings) or implicit feedback (indirect user behavior such as listening, purchasing, watching).

In [0]:
# Meta data about data
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ratings = pd.read_csv("/dbfs/FileStore/df_ratings.csv", header='infer')
display(ratings.head())

movies = pd.read_csv("/dbfs/FileStore/df_movies.csv", header='infer')
display(movies.head())

n_ratings = len(ratings)
n_movies = len(ratings['movieId'].unique())
n_users = len(ratings['userId'].unique())

print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")
print(f"Number of unique users: {n_users}")
print(f"Average ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average ratings per movie: {round(n_ratings/n_movies, 2)}")

print("Sample 10 User v/s the number of Movies they have rated:\n")
user_freq = ratings[['userId', 'movieId']].groupby('userId').count().reset_index()
user_freq.columns = ['userId', 'n_ratings']
print(user_freq.head(10))


userId,movieId,rating,timestamp
1,296,5,1147880044
1,306,3,1147868817
1,307,5,1147868828
1,665,5,1147878820
1,899,3,1147868510


movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


Number of ratings: 25000095
Number of unique movieId's: 59047
Number of unique users: 162541
Average ratings per user: 153.81
Average ratings per movie: 423.39
Sample 10 User v/s the number of Movies they have rated:

 userId n_ratings
0 1 70
1 2 184
2 3 656
3 4 242
4 5 101
5 6 26
6 7 25
7 8 155
8 9 178
9 10 53

In [0]:
# Now, we create user-item matrix using scipy csr matrix
####################################################################################################################################################
from scipy.sparse import csr_matrix
def create_matrix(df):
	
	N = len(df['userId'].unique())
	M = len(df['movieId'].unique())
	
	# Map Ids to indices (userId:Index pair, movierId:Index pair)
	user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
	movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))
	
	# Map indices to IDs (Index:userId pair, Index:movierId pair)
	user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
	movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))
	
	user_index = [user_mapper[i] for i in df['userId']]
	movie_index = [movie_mapper[i] for i in df['movieId']]

	#X=  (Movies_i) X (User_j) --> (Ratings_ij) matrix
    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
	
	return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper
####################################################################################################################################################

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)


#Find Similarity using KNN
####################################################################################################################################################
from sklearn.neighbors import NearestNeighbors
def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
	
	neighbour_ids = []
	
	movie_ind = movie_mapper[movie_id]
	movie_vec = X[movie_ind]
	k+=1
	kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
	kNN.fit(X)
	movie_vec = movie_vec.reshape(1,-1)
	neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
	for i in range(0,k):
		n = neighbour.item(i)
		neighbour_ids.append(movie_inv_mapper[n])
	neighbour_ids.pop(0) #Closest to itself hence exclude that
	return neighbour_ids
####################################################################################################################################################

movie_titles = dict(zip(movies['movieId'], movies['title']))

movie_id = 1 #Movies to find similar from- 'Toy Story'. It will return movies that the similar users(who have seen Toy Story) have watched

similar_ids = find_similar_movies(movie_id, X, k=5)
movie_title = movie_titles[movie_id]

print(f"Since you watched {movie_title}")
for i in similar_ids:
	print(movie_titles[i])


Since you watched Toy Story (1995)
Star Wars: Episode IV - A New Hope (1977)
Toy Story 2 (1999)
Back to the Future (1985)
Forrest Gump (1994)
Star Wars: Episode VI - Return of the Jedi (1983)

In [0]:
movie_titles

Out[28]: {1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'American President, The (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: 'Dangerous Minds (1995)',
 32: 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
 33: 'Wings of Courage (1995)',
 34: 'Babe (1995)',
 35: 'Carrington (1995)',
 36: 'Dead Man Walking (1995)',
 37: 'Across the Sea of Time (1995)',
 38: 'It Takes Two (1995)',
 39: 'Clueless (1995)',
 40: 'Cry, the Beloved Country (1995)',
 41: 'Richard III (1995)',
 42: 'Dead Presidents (1995)',
 43: 'Restoration (1995)',
 44: 'Mortal Kombat (1995)',
 45: 'To Die For (1995)',
 46: 'How to Make an American Quilt (1995)',
 47: 'Seven (a.k.a. Se7en) (1995)',
 48: 'Pocahontas (1995)',
 49: 'When Night Is Falling (1995)',
 50: 'Usual Suspects, The (1995)',
 51: 'Guardian Angel (1994)',
 52: 'Mighty Aphrodite (1995)',
 53: 'Lamerica (1994)',
 54: 'Big Green, The (1995)',
 55: 'Georgia (1995)',
 56: 'Kids of the Round Table (1995)',
 57: 'Home for the Holidays (1995)',
 58: 'Postman, The (Postino, Il) (1994)',
 59: 'Confessional, The (Confessionnal, Le) (1995)',
 60: 'Indian in the Cupboard, The (1995)',
 61: 'Eye for an Eye (1996)',
 62: "Mr. Holland's Opus (1995)",
 63: "Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996)",
 64: 'Two if by Sea (1996)',
 65: 'Bio-Dome (1996)',
 66: 'Lawnmower Man 2: Beyond Cyberspace (1996)',
 67: 'Two Bits (1995)',
 68: 'French Twist (Gazon maudit) (1995)',
 69: 'Friday (1995)',
 70: 'From Dusk Till Dawn (1996)',
 71: 'Fair Game (1995)',
 72: 'Kicking and Screaming (1995)',
 73: 'Misérables, Les (1995)',
 74: 'Bed of Roses (1996)',
 75: 'Big Bully (1996)',
 76: 'Screamers (1995)',
 77: 'Nico Icon (1995)',
 78: 'Crossing Guard, The (1995)',
 79: 'Juror, The (1996)',
 80: 'White Balloon, The (Badkonake sefid) (1995)',
 81: "Things to Do in Denver When You're Dead (1995)",
 82: "Antonia's Line (Antonia) (1995)",
 83: 'Once Upon a Time... When We Were Colored (1995)',
 84: 'Last Summer in the Hamptons (1995)',
 85: 'Angels and Insects (1995)',
 86: 'White Squall (1996)',
 87: 'Dunston Checks In (1996)',
 88: 'Black Sheep (1996)',
 89: 'Nick of Time (1995)',
 90: 'The Journey of August King (1995)',
 92: 'Mary Reilly (1996)',
 93: 'Vampire in Brooklyn (1995)',
 94: 'Beautiful Girls (1996)',
 95: 'Broken Arrow (1996)',
 96: 'In the Bleak Midwinter (1995)',
 97: 'Hate (Haine, La) (1995)',
 98: 'Shopping (1994)',
 99: 'Heidi Fleiss: Hollywood Madam (1995)',
 100: 'City Hall (1996)',
 101: 'Bottle Rocket (1996)',
 102: 'Mr. Wrong (1996)',
 103: 'Unforgettable (1996)',
 104: 'Happy Gilmore (1996)',
 105: 'Bridges of Madison County, The (1995)',
 106: 'Nobody Loves Me (Keiner liebt mich) (1994)',
 107: 'Muppet Treasure Island (1996)',
 108: 'Catwalk (1996)',
 109: 'Headless Body in Topless Bar (1995)',
 110: 'Braveheart (1995)',
 111: 'Taxi Driver (1976)',
 112: 'Rumble in the Bronx (Hont faan kui) (1995)',
 113: 'Before and After (1996)',
 114: "Margaret's Museum (1995)",
 115: 'Happiness Is in the Field (Bonheur est dans le pré, Le) (1995)',
 116: 'Anne Frank Remembered (1995)',
 117: "Young Poisoner's Handbook, The (1995)",
 118: 'If Lucy Fell (1996)',
 119: 'Steal Big, Steal Little 